# Fixing GDPS preciptation problems

The model code requires the hourly input of preciptation, however GDPS files don't have the record in Feb 29th in 2004, 2008 and 2012.

This notebook is trying to fix the problem by using linear interpolation. The code is computational heavy and has been moved to `_script/Modify_GDPS.py`.

In [3]:
import glob
import scipy.io
import numpy as np
import netCDF4 as nc
from __future__ import print_function
% matplotlib inline

In [4]:
ori_obj = nc.Dataset('_data/precip_gdps_y2004.nc', 'r')
ori_obj.variables.keys()

[u'LON', u'LAT', u'TIME', u'precip']

In [6]:
#LON = ori_obj.variables['LON'][:]
#LAT = ori_obj.variables['LAT'][:]
#TIME = ori_obj.variables['TIME'][:]
precip = ori_obj.variables['precip']

(8760, 368, 801)

Feb 29th is the 60th day (1440th hour) in 2004/2008/2012.

In [7]:
print(TIME)

[  1.00000000e+00   2.00000000e+00   3.00000000e+00 ...,   8.75800000e+03
   8.75900000e+03   8.76000000e+03]


In [13]:
TIME = np.arange(1.0, 8762.0, 1.0)

In [ ]:
precip_modified = np.empty([8761, 368, 801])
precip_modified[0:1439, :, :] = precip[0:1439, :, :]
precip_modified[1440, :, :] = (precip[1439, :, :] + precip[1440, :, :])/2.0
precip_modified[1441:8760, :, :] = precip[1440:8759, :, :]

In [14]:
new_obj = nc.Dataset('_data/precip_gdps_y2004_new.nc', 'w') # format='NETCDF4'

time = nc_obj.createDimension('time', None)
lat  = nc_obj.createDimension('lat', 368)
lon  = nc_obj.createDimension('lon', 801)

time_obj = nc_obj.createVariable('TIME', np.float32, ('time',), zlib=True)
lat_obj  = nc_obj.createVariable('LAT', np.float64, ('lat', 'lon'), zlib=True)
lon_obj  = nc_obj.createVariable('LON', np.float64, ('lat', 'lon'), zlib=True)
precip_obj  = nc_obj.createVariable('precip', np.float64, ('time', 'lat', 'lon'), zlib=True)

time_obj[:]=TIME
lon_obj[:] = ori_obj.variables['LON'][:]
lat_obj[:] = ori_obj.variables['LAT'][:]
precip_obj[:] = precip_modified

new_obj.close()

1440